In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display # Allows the use of display() for DataFrames
import seaborn as sb
%matplotlib inline
# from keras.applications.densenet import DenseNet121

import keras
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Input, Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, Add, Concatenate
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
import timeit
import os
from datetime import datetime
import tensorflow as tf
from math import ceil
from enums import TrainingMode, CellType

from main_v2 import (create_multi_generator, build_model, ConvBlock)
from preprocessing import (generate_dataframe_from_csv_horizontal, generate_dataframe_from_csv_vertical, 
                         get_model_inputs)
# https://github.com/keras-team/keras/issues/4161#issuecomment-366031228
# from keras.backend.tensorflow_backend import set_session
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
# config.log_device_placement = True  # to log device placement (on which device the operation ran)
#                                     # (nothing gets printed in Jupyter, only if you run it standalone)
# sess = tf.Session(config=config)
# set_session(sess)  # set this TensorFlow session as the default session for Keras

from train import TrainingRunner

Using TensorFlow backend.


In [2]:

#runner = TrainingRunner(filename="train_root.csv", use_multi=True, epochs=epochs, batch_size=batch_size, val_split=val_split, use_wandb=False)
# runner = TrainingRunner(batch_size=16, filename="train_root.csv", train_mode=TrainingMode.CELL_MULTI)
# runner = TrainingRunner(batch_size=32, train_mode=TrainingMode.SINGLE, use_tb=False, epochs=20)
# runner = TrainingRunner(filename="train_controls_root.csv", batch_size=16, controls_only=True, train_mode=TrainingMode.MULTI)
# runner = TrainingRunner(filename="train_controls_root.csv", batch_size=32, controls_only=True, train_mode=TrainingMode.MULTI, height=224, width=224)
# runner = TrainingRunner(filename="train_controls_root.csv", batch_size=16, controls_only=True, train_mode=TrainingMode.MULTI, weights_path="saved_models/weights.20190821-201844=180=2.hdf5")
# runner = TrainingRunner(filename="train_controls_root.csv", batch_size=16, controls_only=True, train_mode=TrainingMode.MULTI, weights_path="saved_models/weights.20190822-194205=16=2.43.hdf5")
# runner = TrainingRunner(filename="train_controls_root.csv", batch_size=16, controls_only=True, train_mode=TrainingMode.MULTI)
# runner.summary()
# runner = TrainingRunner(filename="test_root.csv", batch_size=32, train_mode=TrainingMode.TEST_ONLY, model_path="saved_models/weights.HUVEC.20190903-075047=284=2.44.hdf5", cell_type=CellType.HUVEC)
runner = TrainingRunner(filename="train_root.csv", batch_size=32, train_mode=TrainingMode.MULTI, model_path="saved_models/weights.HUVEC.20190903-075047=284=2.44.hdf5", cell_type="HEPG2")

0.2 False TrainingMode.MULTI 32 300 HEPG2 224 224 saved_models/weights.HUVEC.20190903-075047=284=2.44.hdf5
FULL DATASET: 1108
creating transfer multi model

W0907 12:30:23.033238 13024 deprecation_wrapper.py:119] From C:\Users\Rbudn\.conda\envs\ml\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0907 12:30:23.056177 13024 deprecation_wrapper.py:119] From C:\Users\Rbudn\.conda\envs\ml\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0907 12:30:23.125990 13024 deprecation_wrapper.py:119] From C:\Users\Rbudn\.conda\envs\ml\lib\site-packages\keras\backend\tensorflow_backend.py:245: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0907 12:30:23.126987 13024 deprecation_wrapper.py:119] From C:\Users\Rbudn\.conda\envs\ml\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0907 12:30:23.126987 13024 depreca

W0907 12:30:23.907899 13024 deprecation_wrapper.py:119] From C:\Users\Rbudn\.conda\envs\ml\lib\site-packages\keras\backend\tensorflow_backend.py:1834: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

W0907 12:30:24.283894 13024 deprecation_wrapper.py:119] From C:\Users\Rbudn\.conda\envs\ml\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0907 12:30:26.528891 13024 deprecation.py:506] From C:\Users\Rbudn\.conda\envs\ml\lib\site-packages\keras\backend\tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0907 12:30:27.975025 13024 deprecation_wrapper.py:119] From C:\Users\Rbudn\.conda\envs\ml\lib\site-packages\keras\optimizers.py:790: The na

In [3]:
runner.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
input_5 (I

In [5]:
for layer in runner.model.layers[:0]:
    layer.trainable = False
    print(layer.name, layer.trainable)

In [3]:
result = runner.predict("test_root.csv")
result

Series([], Name: img_path_root, dtype: object) 0 CellType.HUVEC


[]

In [4]:
# runner.summary()
# runner.run()
# bi, bys = runner.train_generator.__getitem__(1)
# runner.evaluate() 
# [3.5677363468132928, 0.035387431360585725] ['loss', 'acc'] with no checkpoint loaded
# [2.4260314856454177, 0.369737644932705] ['loss', 'acc'] with checkpoint loaded 
# [2.46415597567695, 0.5991458206768804] ['loss', 'acc'] with better model
test = "HEPG2-08_1_B03"
test[:test.find("-")]

'HEPG2'

In [5]:
# df = generate_dataframe_from_csv_horizontal("test.csv", root_name_only=True)
df = pd.read_csv("test_root.csv")
# df
x = df[df["cell_type"] == "HUVEC"]["img_path_root"]
x

8858     train/HUVEC-17/Plate1/B03_s1
8859     train/HUVEC-17/Plate1/B03_s2
8860     train/HUVEC-17/Plate1/B04_s1
8861     train/HUVEC-17/Plate1/B04_s2
8862     train/HUVEC-17/Plate1/B05_s1
                     ...             
26545    train/HUVEC-24/Plate4/O21_s2
26546    train/HUVEC-24/Plate4/O22_s1
26547    train/HUVEC-24/Plate4/O22_s2
26548    train/HUVEC-24/Plate4/O23_s1
26549    train/HUVEC-24/Plate4/O23_s2
Name: img_path_root, Length: 17692, dtype: object

In [5]:
# df["cell_type"] = df["id_code"].apply(lambda r: r[:r.find("-")])
# import pandas as pd
# df = pd.read_csv("test.csv")
# df = pd.read_csv("train_root.csv", dtype={'sirna': object})
# df[df['cell_type'] == test.value]

In [13]:
time_stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
cell_type = "ALL"
checkpoint_path = f"saved_models/model.{cell_type}.weights.{time_stamp}"
checkpoint_path

'saved_models/model.ALL.weights.20190906-072526'

In [9]:
df

,id_code,experiment,plate,well,img_path_root,cell_type
0,HEPG2-08_1_B03,HEPG2-08,1,B03,train/HEPG2-08/Plate1/B03_s1,HEPG2
1,HEPG2-08_1_B03,HEPG2-08,1,B03,train/HEPG2-08/Plate1/B03_s2,HEPG2
2,HEPG2-08_1_B04,HEPG2-08,1,B04,train/HEPG2-08/Plate1/B04_s1,HEPG2
3,HEPG2-08_1_B04,HEPG2-08,1,B04,train/HEPG2-08/Plate1/B04_s2,HEPG2
4,HEPG2-08_1_B05,HEPG2-08,1,B05,train/HEPG2-08/Plate1/B05_s1,HEPG2
5,HEPG2-08_1_B05,HEPG2-08,1,B05,train/HEPG2-08/Plate1/B05_s2,HEPG2
6,HEPG2-08_1_B06,HEPG2-08,1,B06,train/HEPG2-08/Plate1/B06_s1,HEPG2
7,HEPG2-08_1_B06,HEPG2-08,1,B06,train/HEPG2-08/Plate1/B06_s2,HEPG2
8,HEPG2-08_1_B07,HEPG2-08,1,B07,train/HEPG2-08/Plate1/B07_s1,HEPG2
9,HEPG2-08_1_B07,HEPG2-08,1,B07,train/HEPG2-08/Plate1/B07_s2,HEPG2


In [7]:
# df.to_csv("test_root.csv", index=False)

In [5]:
MEANS = np.array([6.74696984, 14.74640167, 10.51260864, 10.45369445,  5.49959796, 9.81545561])
STDS = np.array([7.95876312, 12.17305868, 5.86172946, 7.83451711, 4.701167, 5.43130431])

In [ ]:
# runner.train_generator.__getitem__(1)
runner.run()

checkpoint path:  saved_models/weights.20190827-192518
Instructions for updating:
Use tf.cast instead.
Epoch 1/300
 43/410 [==>...........................] - ETA: 3:49 - loss: 4.2635 - acc: 0.0363

In [1]:
runner.model.save("saved_models\controls.model2.h5")
runner.model.save_weights("saved_models\controls.weights2.h5")
# runner.model.save("saved_models\cell_model.h5")

NameError: name 'runner' is not defined

In [3]:
# wandb.init(project="testing-ml")

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, 
                                   verbose=1, mode='auto', min_delta=0.0001)

early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=15)

csv_logger = CSVLogger(filename='./training_log.csv',
                       separator=',',
                       append=True)

logdir = os.path.join("logs", datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)


(images, trainY) = get_model_inputs(df)

train_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=val_split)

# train_generator = create_multi_generator(df, train_datagen, "training") 
# valid_generator = create_multi_generator(df, train_datagen, "validation") 
train_generator = train_datagen.flow_from_dataframe(
        df,
        directory="./",
        x_col="img_path",
        y_col="sirna",
        target_size=(224, 224),
        batch_size=batch_size,
        subset="training",
        class_mode='categorical')

valid_generator = train_datagen.flow_from_dataframe(
        df,
        directory="./",
        x_col="img_path",
        y_col="sirna",
        target_size=(224, 224),
        batch_size=batch_size,
        subset="validation",
        class_mode='categorical')

Found 54773 validated image filenames belonging to 1108 classes.
Found 18257 validated image filenames belonging to 1108 classes.


In [4]:
model = build_model()
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      4864      
_________________________________________________________________
batch_normalization_1 (Batch (None, 224, 224, 64)      256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
batch_normalization_2

In [ ]:
#https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)
# WORKERS = 2

model.fit_generator(train_generator, 
                    steps_per_epoch=steps,
                    validation_data=valid_generator,
                    validation_steps=steps_valid,
#                     workers=WORKERS, 
#                     use_multiprocessing=True,
                    epochs=epochs, callbacks=[checkpointer,reduceLROnPlat, early, csv_logger, tensorboard_callback], verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
1712/1712 [==============================] - 678s 396ms/step - loss: 7.5768 - acc: 0.0012 - val_loss: 16.4520 - val_acc: 8.2160e-04

Epoch 00001: val_loss improved from inf to 16.45205, saving model to saved_models/weights.best.from_scratch.hdf5
Epoch 2/20
  41/1712 [..............................] - ETA: 9:26 - loss: 7.3360 - acc: 0.0023

In [2]:
# df = generate_dataframe_from_csv_vertical("train.csv", inputs=1)
# df = generate_dataframe_from_csv_horizontal("train.csv")
df = pd.read_csv("./train_controls_root.csv",  dtype={'sirna': object})
val_split = 0.25
epochs = 20
batch_size = 8
train_size = int(len(df)*(1-val_split))
valid_size = int(len(df)*(val_split))
steps = ceil(train_size/batch_size)
steps_valid = ceil(valid_size/batch_size)
print(train_size, valid_size, steps, steps_valid)
# y = df["sirna"]
# display(df)

6145 2048 769 256
